In [1]:
import rasterio
from rasterio.windows import Window
import os



In [2]:
# --- Inputs ---
input_tif = "N.tif"
output_dir = "N_tiles"
tile_size = 256  # pixels per tile edge

os.makedirs(output_dir, exist_ok=True)

with rasterio.open(input_tif) as src:
    width = src.width
    height = src.height

    for i in range(0, width, tile_size):
        for j in range(0, height, tile_size):
            window = Window(i, j, tile_size, tile_size)
            transform = src.window_transform(window)
            tile_data = src.read(window=window)
           
            out_path = os.path.join(output_dir, f"tile_{i}_{j}.tif")
            profile = src.profile
            profile.update({
                "height": tile_data.shape[1],
                "width": tile_data.shape[2],
                "transform": transform
            })
           
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(tile_data)

In [6]:
# renaming files in input folder
import os

def rename_files(folder_path, type):
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    for i, filename in enumerate(files, 1):
        ext = os.path.splitext(filename)[1]
        new_name = f"tile_{i}.{type}"
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_name))


In [ ]:
# rename_files("N_pngs", "png")

In [3]:
import os
from PIL import Image

def convert_tif_to_png(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".tif"):
            img = Image.open(os.path.join(input_dir, filename))
            new_name = os.path.splitext(filename)[0] + ".png"
            img.save(os.path.join(output_dir, new_name))


In [ ]:
# convert_tif_to_png("N_tiles", "N_pngs")